# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
# Import API key
from api_keys import g_key

import warnings
warnings.filterwarnings('ignore')

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv", index_col=0)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,56.28,76,100,5.99,CA,1628233763
1,Longyearbyen,78.2186,15.6401,46.24,66,75,2.30,SJ,1628233764
2,Antalaha,-14.9003,50.2788,74.79,61,51,10.67,MG,1628233764
3,Menongue,-14.6585,17.6910,64.62,21,0,6.17,AO,1628233764
4,Bredasdorp,-34.5322,20.0403,56.28,91,89,4.38,ZA,1628233765


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#cities_short = cities_df[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70)], 
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 75) & 
               (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780,
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827,
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860,
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862,
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873,
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885,
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934,


In [11]:
params = {
    "radius": 5000,
    "types": "hotel",
    #"keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    pprint.pprint(hotel_name)
    
#     print(index)
#     print(row)

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{'html_attributions': [],
 'next_page_token': 'Aap_uEBrCXWj42O5Sjc07cEzQK1obkp_sAiCU-mWVb9hy6VOsoGqTFJlY3UZAkCAn5n93BihfwQfr9j9fpOFhm4aQjjwhFh1tzuXmne2CUzplULDWorgm0cmW9bSbMEbSm14AKaPxdNgUcW83uTQsJidbC4lzOeUAr9XXiIHkroWNiNr6YgBHN9I6h-g67yzUbsLmNkSDwSRCrx9eN71sBGnEXNNYoHmCQGoOIeLmCwotzAu-ZUbXzn26mG8fNLO8hF4cy_0Hy2HTXuzSAod1okjidQUI73r5yB4o90BBd6Rp4lLNamvraCJMWsi4rfDx5vl0RJX_AV6z39AaxPKlPyQ78ePbfdFnozFsWL5iRsKLeAtWJKcDi3to64sNyiZpcsytGIu-pCKkjOjMVtgyuh2ycj_revu9SGNSFaOP4SNuY3uGU6z5KSihQ',
 'results': [{'geometry': {'location': {'lat': -1.7762778, 'lng': -48.7891012},
                           'viewport': {'northeast': {'lat': -1.765125138810075,
                                                      'lng': -48.77309377622858},
                                        'southwest': {'lat': -1.787430393867588,
                                                      'lng': -48.80510862377142}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',


{'html_attributions': [],
 'next_page_token': 'Aap_uEAnHsGDoqf7Rx81GdyZV3jnBQLxfOSYYPIreqBLpvzRMGjYcr4b_LjKpAzjpHT9WwzLp0_JJINUD3dAb8FPY6uDxRnMN3ARcDqy8AG5GZqEJqqTbkwCf89rjh3qcYefP3CXyx7O0ma21Dl5Oh21ROv9n-9avbuJJnHQdNtKvDyb6zu-xRTP70gAFXEJpcIGExAcMCjyNpVZH_h6pE19EHIbqz9Eu8DwcJq7scFdMG4UgKwilIIuyxnjobjsj4CRg6hMRocp0BuaVWtErzYlcy1LDeRMF8dlpJ02m7jlGY7ir4QfzuD9fDUMi0Ap-LZ5YZvSeKMoIJLXi7gpGkFZZI3CqNsZ8tmTk4n1FMPbI0e-Ej0eSJx4CwLqi90JfZCxKv8OXQCVkqkOlNMkyduIg-eOFycBxWo3rCYAg-GN6-5imaw7-MZEWA',
 'results': [{'geometry': {'location': {'lat': 45.4732537, 'lng': 10.8507277},
                           'viewport': {'northeast': {'lat': 45.48025503502558,
                                                      'lng': 10.88168337120799},
                                        'southwest': {'lat': 45.4389847325092,
                                                      'lng': 10.83754916623245}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
      

{'html_attributions': [],
 'next_page_token': 'Aap_uEDqyGCUAIJOWhqdwW0MNTrmPlt_HjW7aTQk2yoYqg1hE71_3ZxngB9dTE_e92tRhqrfPFzGEuqf7gtEBCl7RvUVzrKL4uuY7hM57fhoI1oFg0v5ntV7USXiMEZE1LIyd1NVhLayCkhN75GV9gWVNVIrT6URT-maJS6VE6bVFwTPlvfDIRCS259Z3h2aiUNs6b72ClDlDlSYEf-o-OBmC4JvPQhz_d9mStnT_VXO_yeiymreYl0GgcM5_SIy-eI0Tryf1ghpAX-IBSPniXDBPUL64odnhETYFDoqnqK4McqO7Xft1F3EknjU9TiYllj0qH-iLPwXvj4TKmAYMFFBjUWxHAWXZd11zDoEHUnxFFnGaURcxQFbnb7unmvYTvaGGQYK06Yv7I2rjnG_sAqzTz0Ttl_KlE2NYhRlDBVwWJexJeMpwEaSAQ',
 'results': [{'geometry': {'location': {'lat': 63.74644029999999,
                                        'lng': 121.6276666},
                           'viewport': {'northeast': {'lat': 63.76156902577657,
                                                      'lng': 121.6604761223102},
                                        'southwest': {'lat': 63.73641103858803,
                                                      'lng': 121.5999091141373}}},
              'icon': 'https://maps.gstatic.com/mapfiles/

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -5.8871908, 'lng': 35.2202016},
                           'viewport': {'northeast': {'lat': -5.864244347213863,
                                                      'lng': 35.23306846524302},
                                        'southwest': {'lat': -5.895663782217955,
                                                      'lng': 35.19804945450468}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'icon_background_color': '#7B9EB0',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
              'name': 'Kintinku',
              'place_id': 'ChIJz7nEw1UHTRgRRHNFYBU1Dt0',
              'reference': 'ChIJz7nEw1UHTRgRRHNFYBU1Dt0',
              'scope': 'GOOGLE',
              'types': ['locality', 'political'],
              'vicinity': 'Kintinku'},
             {'business_status': '

{'html_attributions': [],
 'next_page_token': 'Aap_uEDeTpYVDzTQQDWUwFds1m7zLhgwK2N-wJ4mj9LTKPYsEJpqnN0QkucJxnC3l5kQ1c6q7fAahUlsJl5N23GHRjUT7FVVNClQUsLWGUXl9OH8068rfRMDBc7YfnHo3Rog0SBe8A_1y_E4iLz3Qn91h6Df27AaR9aHybWCJwUWyOBYevFjikqWLOlOrFktonXJ4sqM6Gynz2UIaVd5iOIBf23kaxwRHFh4GD2Oj3aa9rXOhsi480j5fN_Bc7zSIFgVL3rDbYUOkd84XA2gxDzTXOBtf30Z_-n_mEF7dquzICPY-fIDswJoG4ApiSfwtCI1gW7jwFTxdEG8i1sadZphtAxs3tklCgU1qHvWDsL-cvquBF-Vz4XF2Cym1hx38NQYe2FM9taa8DBvg2EgI87oHMD7N0s1iEA00E7eU8VMp5wgwQrpeO--gQ',
 'results': [{'geometry': {'location': {'lat': -28.7750917, 'lng': 114.609858},
                           'viewport': {'northeast': {'lat': -28.76276761545247,
                                                      'lng': 114.6261532804849},
                                        'southwest': {'lat': -28.78861926424706,
                                                      'lng': 114.5962706435342}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  

{'html_attributions': [],
 'next_page_token': 'Aap_uECekzBOSuf6m6q0UQLxLG6A_at8oE3JNeAk_TXF7hTAph2-6RKVdns2JL7nau3hXpxlA81-5pZmKjeZSlrhDczcKyXsBtqExsIkmGmcMb1Uj-6sHmUG0XkqqN7NsNTT2gj4qhHCfbfpi1H1pkPyDOMBfn-NfVqiBTjLn24sHaFrZOORos8LmxJYD7psHIcK1lnf2Tys5TYPOYxe9VIbz70jdIn35frHHiNrxiC8Rl_outtrh8HwY9_piuUDxtQto0W8O84U2mtsw_7pKGa-BMYl7CKpOSwEt5GoZnCGtXMr8KlHeZJOJNlDGWH4yNMFI3a7nvVeXckIRrS2N4D2NqmVjKjLN7KMC9r1mr6LBUbjDU3voTHK67rU0lxRh8UkacigQRrFNk3S80bY3OqdQkm_2fgI_GXrcuDDQun5ODJ9A4uCOjk6aQ',
 'results': [{'geometry': {'location': {'lat': 60.0612676, 'lng': 71.453642},
                           'viewport': {'northeast': {'lat': 60.07314507800471,
                                                      'lng': 71.47693214193717},
                                        'southwest': {'lat': 60.05229792722047,
                                                      'lng': 71.43950217649338}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
      

{'html_attributions': [],
 'next_page_token': 'Aap_uEBtbI_3E8mBboM3A4SVgVjojrtzZcvkBIx5CmEnRL51daK22Z0thEwNoxoccpjPo2QemJ2-XQkpFxR3Tjd5tQBcLLr3vkbtRjvciCVgAteEa8i2N7JBsEs8S-mKgee9CgyvXjnRdq5GAgy4ytN_Kcsk012bO7pcUNLEaM3kdiDp766Hp2yx3z8NQ7t0_8NEDCFQwT9AKZ7dWo7EOmy5kKlMzFA75MJUFFXWLGm6N3ENnoDHvBRVK8m6XD_Mt39laGXP1C2QOdMK7HUx5EAg0vv_PrF5TrAtnYKqsTqliEYVqYY1Hj15fZXXHdWcogfoIsaRZy5M5jLm9hO88Qp1VKlXWguVCQRHzU211iPJ_nkB-yelvKQIn3-G5hJ7Gz9q0m2sCIrzmPJXcdpABRIkMvl-WOOHyFYO-jOgVbz-44NrgZhDCZjVKA',
 'results': [{'geometry': {'location': {'lat': -11.7172157, 'lng': 43.2473146},
                           'viewport': {'northeast': {'lat': -11.67423352854021,
                                                      'lng': 43.27753332745275},
                                        'southwest': {'lat': -11.72766405347954,
                                                      'lng': 43.23973093603298}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  

In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,Ipixuna,-1.7625,-48.8043,76.95,95,0,3.22,BR,1628233780,Colônia Velha
185,Pescantina,45.4837,10.8685,75.49,77,0,1.14,IT,1628233827,Bussolengo
281,Vilyuysk,63.7553,121.6247,78.80,26,0,4.79,RU,1628233860,Vilyuysk
286,Kintinku,-5.8833,35.2333,75.20,48,0,6.98,TZ,1628233862,Kintinku
317,Geraldton,-28.7667,114.6000,75.13,43,0,8.32,AU,1628233873,Geraldton
352,Salym,60.0625,71.4789,77.09,42,0,4.09,RU,1628233885,Salym
499,Moroni,-11.7022,43.2551,77.40,61,0,4.09,KM,1628233934,Moroni


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map


# Display figure
